<a href="https://colab.research.google.com/github/Zheng7426/machine_learning_practice/blob/master/CNN_assignment1_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Upload The cifar10 from the keras.datasets. The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, 
with 6000 images per class. There are 50000 training images and 10000 test images. 

for more information about the dataset please see: https://www.cs.toronto.edu/~kriz/cifar.html

Split data to the train and test.
Convert the class label to binary class label
## Assignment 3
### Zheng Huang 001430398

In [0]:
import numpy as np

from keras.layers import Input,Activation,Dropout,LeakyReLU,AveragePooling2D,Flatten,Dense,Conv2D,BatchNormalization,MaxPooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K

from keras.datasets import cifar10


In [0]:
# Load the dataset
num_classes = 10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [0]:
x_train.shape

(50000, 32, 32, 3)

Normalized the data and train the model using CNN.

Pixel values for each image in the dataset are integers in the range between 0 and 255(no color and full color).


In [0]:
# Rescale the pixel values to the range[0,1]
# Convert from integer to floats
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Converts a class vector (integers) to binary class matrix, 
# transforming the class interger to into a 10-element binary vector with a 1 for the index of the class
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)



Model structure:<br>
BatchNormalization is a technique which subtracts the weights from its mean and then divide by standard deviation. It prevents the weights to go off the rails. <br>
LeakyReLU is to add non-linearity to the system.<br>
MaxPooling2D is to reduce the dimension of original layer by selecting the max values in an area of data.<br>
Dropout prevents overfitting.<br>
After flatten layer, the Dense layer fully connects all the weights with each other.


In [0]:
input_layer = Input((32,32,3))

x = Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = 'same')(input_layer)
x = BatchNormalization()(x)
x = LeakyReLU()(x)


x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization(momentum=0.9)(x)
x = LeakyReLU()(x)




x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)


x = Conv2D(filters = 64, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization(momentum=0.9)(x)
x = LeakyReLU()(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(rate = 0.2)(x)



x = Conv2D(filters = 82, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization(momentum=0.9)(x)
x = LeakyReLU()(x)



x = Conv2D(filters = 82, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization(momentum=0.9)(x)
x = LeakyReLU()(x)



x = Flatten()(x)

x = Dense(128)(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(rate = 0.5)(x)

x = Dense(num_classes)(x)
output_layer = Activation('softmax')(x)

model = Model(input_layer, output_layer)


In [0]:
model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_66 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
leaky_re_lu_66 (LeakyReLU)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 16, 16, 32)        9248      
_________________________________________________________________
batch_normalization_67 (Batc (None, 16, 16, 32)        128       
_________________________________________________________________
leaky_re_lu_67 (LeakyReLU)   (None, 16, 16, 32)        0  

In [0]:
opt = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [0]:
model.fit(x_train
         , y_train
         , batch_size=32
         , epochs=20
         , shuffle=True
         , validation_data = (x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 48s 958us/step - loss: 1.6241 - acc: 0.4414 - val_loss: 1.1925 - val_acc: 0.5666
Epoch 2/20
50000/50000 [==============================] - 42s 835us/step - loss: 1.1567 - acc: 0.5887 - val_loss: 1.0365 - val_acc: 0.6295
Epoch 3/20
50000/50000 [==============================] - 43s 851us/step - loss: 1.0069 - acc: 0.6472 - val_loss: 0.9750 - val_acc: 0.6557
Epoch 4/20
50000/50000 [==============================] - 42s 834us/step - loss: 0.9132 - acc: 0.6806 - val_loss: 0.8576 - val_acc: 0.7067
Epoch 5/20
50000/50000 [==============================] - 41s 827us/step - loss: 0.8448 - acc: 0.7070 - val_loss: 0.8183 - val_acc: 0.7177
Epoch 6/20
50000/50000 [==============================] - 42s 840us/step - loss: 0.7836 - acc: 0.7269 - val_loss: 0.8196 - val_acc: 0.7130
Epoch 7/20
50000/50000 [==============================] - 42s 847us/step - loss: 0.7340 - acc: 0.7473 - val_loss: 0.

Evaluate the accuracy of the model. The first value is for loss. The second value returned is accuracy of the model regarding test set.

In [0]:
model.evaluate(x_test, y_test, batch_size=1000)

10000/10000 [==============================] - 1s 83us/step


[0.6569806158542633, 0.7861000001430511]